In [1]:
train_path = "train.csv"
test_path = "test.csv"
label = "label"
model_file = "model.pkl"
#prediction_label_column = ""

In [2]:
# Define available checks (without parameters)
check_options = {
    "IsSingleValue": IsSingleValue(),
    "SpecialCharacters": SpecialCharacters(),
    "MixedNulls": MixedNulls(),
    "MixedDataTypes": MixedDataTypes(),
    "StringMismatch": StringMismatch(),  # Needs parameter
    "DataDuplicates": DataDuplicates(),
    "StringLengthOutOfBounds": StringLengthOutOfBounds(),
    "ConflictingLabels": ConflictingLabels(),
    "OutlierSampleDetection": OutlierSampleDetection(),  # Needs parameter
    "FeatureLabelCorrelation": FeatureLabelCorrelation(),
    "FeatureFeatureCorrelation": FeatureFeatureCorrelation(),  # Needs parameter
    "IdentifierLabelCorrelation": IdentifierLabelCorrelation()
}

# Checks that require user input parameters
checks_with_params = {
    "StringMismatch": ["similarity_threshold"],
    "OutlierSampleDetection": ["max_outliers_ratio", "outlier_score_threshold"],
    "FeatureFeatureCorrelation": ["correlation_threshold"]
}

# Checks that require user input parameters
checks_with_params = {
    "StringMismatch": {"similarity_threshold":5},
    "OutlierSampleDetection": {"max_outliers_ratio": 0.1, "outlier_score_threshold":5},
    "FeatureFeatureCorrelation": {"correlation_threshold":0.6}
}



NameError: name 'IsSingleValue' is not defined

In [3]:
# Display available checks
print("\nAvailable Data Integrity Checks:")
for i, test in enumerate(check_options.keys()):
    print(f"{i + 1}. {test}")

# User selects which checks to run
selected_indices = input("Enter check numbers to run (comma-separated, e.g., 1,3,5): ")
selected_indices = [int(idx.strip()) - 1 for idx in selected_indices.split(',') if idx.strip().isdigit()]

# Map indices to selected check names
selected_tests = [list(check_options.keys())[i] for i in selected_indices]

# Get threshold values only for selected tests that require them
params = get_threshold_values(selected_tests)
params


Available Data Integrity Checks:


NameError: name 'check_options' is not defined

In [4]:
import deepchecks
from deepchecks.tabular import Suite
from deepchecks.tabular import Dataset
from deepchecks.tabular.checks import (
    IsSingleValue, SpecialCharacters, MixedNulls, MixedDataTypes, StringMismatch, DataDuplicates,
    StringLengthOutOfBounds, ConflictingLabels, OutlierSampleDetection, FeatureLabelCorrelation,
    FeatureFeatureCorrelation, IdentifierLabelCorrelation
)

def run_deepchecks(data1, data2, label=None):
    """Run selected deepchecks data integrity tests on the given dataset."""
    dataset1 = Dataset(data1, label=label)
    dataset2 = Dataset(data2, label=label)
        
    # Create selected checks with conditions where applicable
    selected_checks = []
    for test in selected_tests:
        if test == "StringMismatch":
            #check = StringMismatch(similarity_threshold=params['similarity_threshold']).add_condition_number_variants_less_or_equal(5)
            check = StringMismatch().add_condition_number_variants_less_or_equal(1)
        elif test == "OutlierSampleDetection":
            check = OutlierSampleDetection().add_condition_outlier_ratio_less_or_equal(
                params['max_outliers_ratio'], params['outlier_score_threshold']
            )
        elif test == "FeatureFeatureCorrelation":
            check = FeatureFeatureCorrelation(correlation_threshold=params['correlation_threshold']).add_condition_max_number_of_pairs_above_threshold(
                threshold=params['correlation_threshold'], n_pairs=0
            )
        elif test == "IsSingleValue":
            check = IsSingleValue().add_condition_not_single_value()
        elif test == "SpecialCharacters":
            check = SpecialCharacters().add_condition_ratio_of_special_characters_less_or_equal(max_ratio = 0.001)  # No condition available for this check   
        elif test == "MixedNulls":
            check = MixedNulls().add_condition_different_nulls_less_equal_to()
        elif test == "MixedDataTypes":
            check = MixedDataTypes().add_condition_rare_type_ratio_not_in_range((0.01, 0.1))
        elif test == "DataDuplicates":
            check = DataDuplicates().add_condition_ratio_less_or_equal(max_ratio=0.05)
        elif test == "ConflictingLabels":
            check = ConflictingLabels().add_condition_ratio_of_conflicting_labels_less_or_equal(max_ratio=0)
        elif test == "FeatureLabelCorrelation":
            check = FeatureLabelCorrelation().add_condition_feature_pps_less_than(threshold=0.8)
        elif test == "IdentifierLabelCorrelation":
            check = IdentifierLabelCorrelation().add_condition_pps_less_or_equal(max_pps=0)
        else:
            check = check_options[test]

        selected_checks.append(check)
    
    # Create and run suite
    suite = Suite("Custom Data Integrity Suite", *selected_checks)
    result = suite.run(dataset1, dataset2)

    # Show results
    result.show()
    return result


def load_csv(file_path):
    """Load CSV file into a Pandas DataFrame."""
    try:
        df = pd.read_csv(file_path)
        if df.empty:
            raise ValueError("Uploaded file is empty.")
        print(f"Loaded {file_path} with shape: {df.shape}")
        return df
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

# Example usage
if __name__ == "__main__":
    import pandas as pd
    print("Loading data...")
    train_df = load_csv(TRAIN_DATA_PATH)
    test_df = load_csv(TEST_DATA_PATH)    
    # Load or create a sample dataset
    
    run_deepchecks(train_df, test_df)


deepchecks - WARNING - You are using deepchecks version 0.19.0, however a newer version is available. Deepchecks is frequently updated with major improvements. You should consider upgrading via the "python -m pip install --upgrade deepchecks" command.


Loading data...


NameError: name 'TRAIN_DATA_PATH' is not defined

In [15]:
import deepchecks
from deepchecks.tabular import Suite
from deepchecks.tabular import Dataset
from deepchecks.tabular.checks import (
    IsSingleValue, SpecialCharacters, MixedNulls, MixedDataTypes, StringMismatch, DataDuplicates,
    StringLengthOutOfBounds, ConflictingLabels, OutlierSampleDetection, FeatureLabelCorrelation,
    FeatureFeatureCorrelation, IdentifierLabelCorrelation
)

def get_threshold_values(selected_tests):
    """Ask for threshold values only for selected tests that require them."""
    params = {}
    for test in selected_tests:
        if test in checks_with_params:
            for param in checks_with_params[test]:
                params[param] = float(input(f"Enter {param} for {test}: "))
    print(params)
    return params

def run_deepchecks(data1, data2, label=None):
    """Run selected deepchecks data integrity tests on the given dataset."""
    dataset1 = Dataset(data1, label=label)
    dataset2 = Dataset(data2, label=label)
    # Display available checks
    print("\nAvailable checks:")
    for i, test in enumerate(check_options.keys()):
        print(f"{i + 1}. {test}")

    # User selects which checks to run
    selected_indices = input("Enter check numbers to run (comma-separated, e.g., 1,3,5): ")
    selected_indices = [int(idx.strip()) - 1 for idx in selected_indices.split(',') if idx.strip().isdigit()]
    
    # Map indices to selected check names
    selected_tests = [list(check_options.keys())[i] for i in selected_indices]
    
    # Get threshold values only for selected tests that require them
    params = get_threshold_values(selected_tests)
    
    # Create selected checks with conditions where applicable
    selected_checks = []
    for test in selected_tests:
        if test == "StringMismatch":
            #check = StringMismatch(similarity_threshold=params['similarity_threshold']).add_condition_number_variants_less_or_equal(5)
            check = StringMismatch().add_condition_number_variants_less_or_equal(1)
        elif test == "OutlierSampleDetection":
            check = OutlierSampleDetection().add_condition_outlier_ratio_less_or_equal(
                params['max_outliers_ratio'], params['outlier_score_threshold']
            )
        elif test == "FeatureFeatureCorrelation":
            check = FeatureFeatureCorrelation(correlation_threshold=params['correlation_threshold']).add_condition_max_number_of_pairs_above_threshold(
                threshold=params['correlation_threshold'], n_pairs=0
            )
        elif test == "IsSingleValue":
            check = IsSingleValue().add_condition_not_single_value()
        elif test == "SpecialCharacters":
            check = SpecialCharacters().add_condition_ratio_of_special_characters_less_or_equal(max_ratio = 0.001)  # No condition available for this check   
        elif test == "MixedNulls":
            check = MixedNulls().add_condition_different_nulls_less_equal_to()
        elif test == "MixedDataTypes":
            check = MixedDataTypes().add_condition_rare_type_ratio_not_in_range((0.01, 0.1))
        elif test == "DataDuplicates":
            check = DataDuplicates().add_condition_ratio_less_or_equal(max_ratio=0.05)
        elif test == "ConflictingLabels":
            check = ConflictingLabels().add_condition_ratio_of_conflicting_labels_less_or_equal(max_ratio=0)
        elif test == "FeatureLabelCorrelation":
            check = FeatureLabelCorrelation().add_condition_feature_pps_less_than(threshold=0.8)
        elif test == "IdentifierLabelCorrelation":
            check = IdentifierLabelCorrelation().add_condition_pps_less_or_equal(max_pps=0)
        else:
            check = check_options[test]

        selected_checks.append(check)
    
    # Create and run suite
    suite = Suite("Custom Data Integrity Suite", *selected_checks)
    result = suite.run(dataset1, dataset2)

    # Show results
    result.show()
    return result


def load_csv(file_path):
    """Load CSV file into a Pandas DataFrame."""
    try:
        df = pd.read_csv(file_path)
        if df.empty:
            raise ValueError("Uploaded file is empty.")
        print(f"Loaded {file_path} with shape: {df.shape}")
        return df
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

# Example usage
if __name__ == "__main__":
    import pandas as pd
    print("Loading data...")
    train_df = load_csv(TRAIN_DATA_PATH)
    test_df = load_csv(TEST_DATA_PATH)    
    # Load or create a sample dataset
    
    run_deepchecks(train_df, test_df)


deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 2 categorical features were inferred.: feature3, label
deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 2 categorical features were inferred.: feature3, label


Loading data...
Loaded train.csv with shape: (1000, 4)
Loaded test.csv with shape: (1000, 4)

Available checks:
1. IsSingleValue
2. SpecialCharacters
3. MixedNulls
4. MixedDataTypes
5. StringMismatch
6. DataDuplicates
7. StringLengthOutOfBounds
8. ConflictingLabels
9. OutlierSampleDetection
10. FeatureLabelCorrelation
11. FeatureFeatureCorrelation
12. IdentifierLabelCorrelation


Enter check numbers to run (comma-separated, e.g., 1,3,5):  1,9
Enter max_outliers_ratio for OutlierSampleDetection:  0.1
Enter outlier_score_threshold for OutlierSampleDetection:  0.9


{'max_outliers_ratio': 0.1, 'outlier_score_threshold': 0.9}


Accordion(children=(VBox(children=(HTML(value='\n<h1 id="summary_R421CEOGRYFUJPFT46L536AKX">Custom Data Integr…

In [5]:
import deepchecks
#from deepchecks.core.suite import Suite
from deepchecks.tabular import Suite
from deepchecks.tabular import Dataset
from deepchecks.tabular.checks import (
    IsSingleValue, SpecialCharacters, MixedNulls, OutlierSampleDetection
)

def get_user_input():
    """Get user-defined threshold values for Outlier Sample Detection."""
    while True:
        try:
            outlier_score_threshold = float(input("Enter Outlier Score Threshold (e.g., 3.0): "))
            max_outliers_ratio = float(input("Enter Max Outliers Ratio (e.g., 0.1): "))
            if 0 <= max_outliers_ratio <= 1:
                return outlier_score_threshold, max_outliers_ratio
            else:
                print("Max Outliers Ratio must be between 0 and 1.")
        except ValueError:
            print("Invalid input. Please enter numeric values.")

def run_data_integrity(data, label=None):
    """Run selected deepchecks data integrity tests on the given dataset."""
    dataset = Dataset(data, label=label)
    
    # Get threshold values for Outlier Sample Detection
    outlier_score_threshold, max_outliers_ratio = get_user_input()
    
    # Define checks
    checks = [
        IsSingleValue(),
        SpecialCharacters(),
        MixedNulls(),
        OutlierSampleDetection()
        .add_condition_outlier_ratio_less_or_equal(max_outliers_ratio, outlier_score_threshold)
    ]
    
    # Create and run suite
    #suite = Suite("Custom Data Integrity Suite", checks)
    suite = Suite("Custom Data Integrity Suite", *checks)
    result = suite.run(dataset)
    
    # Show results
    result.show()
    return result


# Example usage
import pandas as pd

# Load or create a sample dataset
data = pd.DataFrame({
    'feature1': [1, 1, 1, 1, 1],  # Single value column
    'feature2': ['@hello', 'world!', 'test123', 'data$', 'valid'],  # Special characters
    'feature3': [None, 'NULL', 'N/A', '', 'Missing'],  # Mixed nulls
    'feature4': [1, 100, 200, -50, 3000]  # Outliers
})

run_data_integrity(data)

In [6]:
# Example usage
import pandas as pd

# Load or create a sample dataset
data = pd.DataFrame({
    'feature1': [1, 1, 1, 1, 1],  # Single value column
    'feature2': ['@hello', 'world!', 'test123', 'data$', 'valid'],  # Special characters
    'feature3': [None, 'NULL', 'N/A', '', 'Missing'],  # Mixed nulls
    'feature4': [1, 100, 200, -50, 3000]  # Outliers
})

run_data_integrity(data)

deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 4 categorical features were inferred.: feature1, feature2, feature3, feature4


Enter Outlier Score Threshold (e.g., 3.0):  12
Enter Max Outliers Ratio (e.g., 0.1):  12


Max Outliers Ratio must be between 0 and 1.


Enter Outlier Score Threshold (e.g., 3.0):  12
Enter Max Outliers Ratio (e.g., 0.1):  12


Max Outliers Ratio must be between 0 and 1.


...

Max Outliers Ratio must be between 0 and 1.


Enter Outlier Score Threshold (e.g., 3.0):  3
Enter Max Outliers Ratio (e.g., 0.1):  


Invalid input. Please enter numeric values.


Enter Outlier Score Threshold (e.g., 3.0):  1


KeyboardInterrupt: Interrupted by user

In [3]:
import deepchecks
from deepchecks.tabular import Suite
from deepchecks.tabular import Dataset
from deepchecks.tabular.checks import (
    IsSingleValue, SpecialCharacters, MixedNulls, MixedDataTypes, StringMismatch, DataDuplicates,
    StringLengthOutOfBounds, ConflictingLabels, OutlierSampleDetection, FeatureLabelCorrelation,
    FeatureFeatureCorrelation, IdentifierLabelCorrelation
)

def get_user_input():
    """Get user-defined parameter values for checks that require input."""
    params = {}
    params['outlier_score_threshold'] = float(input("Enter Outlier Score Threshold (e.g., 3.0): "))
    params['max_outliers_ratio'] = float(input("Enter Max Outliers Ratio (e.g., 0.1): "))
    params['similarity_threshold'] = float(input("Enter Similarity Threshold for String Mismatch (e.g., 0.8): "))
    params['correlation_threshold'] = float(input("Enter Correlation Threshold for Feature-Feature Correlation (e.g., 0.9): "))
    return params

def run_deepchecks(data, label=None):
    """Run selected deepchecks data integrity tests on the given dataset."""
    dataset = Dataset(data, label=label)
    
    # Get threshold values for checks requiring parameters
    params = get_user_input()
    
    # Define checks with input parameters where applicable
    checks = [
        IsSingleValue(),
        SpecialCharacters(),
        MixedNulls(),
        MixedDataTypes(),
        StringMismatch(similarity_threshold=params['similarity_threshold']),
        DataDuplicates(),
        StringLengthOutOfBounds(),
        ConflictingLabels(),
        OutlierSampleDetection()
        .add_condition_outlier_ratio_less_or_equal(params['max_outliers_ratio'], params['outlier_score_threshold']),
        FeatureLabelCorrelation(),
        FeatureFeatureCorrelation(correlation_threshold=params['correlation_threshold']),
        IdentifierLabelCorrelation()
    ]
    
    # Allow user to select which checks to run
    print("Available checks:")
    for i, check in enumerate(checks):
        print(f"{i + 1}. {check.name}")
    selected_indices = input("Enter check numbers to run (comma-separated, e.g., 1,3,5): ")
    selected_indices = [int(idx.strip()) - 1 for idx in selected_indices.split(',') if idx.strip().isdigit()]
    
    selected_checks = [checks[i] for i in selected_indices]
    
    # Create and run suite
    suite = Suite("Custom Data Integrity Suite", *selected_checks)
    result = suite.run(dataset)
    
    # Show results
    result.show()
    return result

# Example usage
if __name__ == "__main__":
    import pandas as pd
    
    # Load or create a sample dataset
    data = pd.DataFrame({
        'feature1': [1, 1, 1, 1, 1],  # Single value column
        'feature2': ['@hello', 'world!', 'test123', 'data$', 'valid'],  # Special characters
        'feature3': [None, 'NULL', 'N/A', '', 'Missing'],  # Mixed nulls
        'feature4': [1, 100, 200, -50, 3000]  # Outliers
    })
    
    run_deepchecks(data)


deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 4 categorical features were inferred.: feature1, feature2, feature3, feature4


Enter Outlier Score Threshold (e.g., 3.0):  3
Enter Max Outliers Ratio (e.g., 0.1):  0.2
Enter Similarity Threshold for String Mismatch (e.g., 0.8):  0.8
Enter Correlation Threshold for Feature-Feature Correlation (e.g., 0.9):  0.8


Available checks:
1. <bound method BaseCheck.name of <class 'deepchecks.tabular.checks.data_integrity.is_single_value.IsSingleValue'>>
2. <bound method BaseCheck.name of <class 'deepchecks.tabular.checks.data_integrity.special_chars.SpecialCharacters'>>
3. <bound method BaseCheck.name of <class 'deepchecks.tabular.checks.data_integrity.mixed_nulls.MixedNulls'>>
4. <bound method BaseCheck.name of <class 'deepchecks.tabular.checks.data_integrity.mixed_data_types.MixedDataTypes'>>
5. <bound method BaseCheck.name of <class 'deepchecks.tabular.checks.data_integrity.string_mismatch.StringMismatch'>>
6. <bound method BaseCheck.name of <class 'deepchecks.tabular.checks.data_integrity.data_duplicates.DataDuplicates'>>
7. <bound method BaseCheck.name of <class 'deepchecks.tabular.checks.data_integrity.string_length_out_of_bounds.StringLengthOutOfBounds'>>
8. <bound method BaseCheck.name of <class 'deepchecks.tabular.checks.data_integrity.conflicting_labels.ConflictingLabels'>>
9. <bound method B

Enter check numbers to run (comma-separated, e.g., 1,3,5):  1, 2, 6, 10, 11,12


Accordion(children=(VBox(children=(HTML(value='\n<h1 id="summary_PNFX1MR90FOLXZXPJEY698F8Y">Custom Data Integr…

In [4]:
import deepchecks
from deepchecks.tabular import Suite
from deepchecks.tabular import Dataset
from deepchecks.tabular.checks import (
    IsSingleValue, SpecialCharacters, MixedNulls, MixedDataTypes, StringMismatch, DataDuplicates,
    StringLengthOutOfBounds, ConflictingLabels, OutlierSampleDetection, FeatureLabelCorrelation,
    FeatureFeatureCorrelation, IdentifierLabelCorrelation
)

def get_user_input():
    """Get user-defined parameter values for checks that require input."""
    params = {}
    params['outlier_score_threshold'] = 3.0 #float(input("Enter Outlier Score Threshold (e.g., 3.0): "))
    params['max_outliers_ratio'] = 0.1 #float(input("Enter Max Outliers Ratio (e.g., 0.1): "))
    params['similarity_threshold'] = 0.8 #float(input("Enter Similarity Threshold for String Mismatch (e.g., 0.8): "))
    params['correlation_threshold'] = 0.9 #float(input("Enter Correlation Threshold for Feature-Feature Correlation (e.g., 0.9): "))
    return params

def run_deepchecks(data, label=None):
    """Run selected deepchecks data integrity tests on the given dataset."""
    dataset = Dataset(data, label=label)
    
    # Get threshold values for checks requiring parameters
    params = get_user_input()
    
    # Define checks with valid conditions where applicable
    checks = [
        IsSingleValue().add_condition_not_single_value(),  # Ensure column is not a single value
        SpecialCharacters().add_condition_ratio_of_special_characters_less_or_equal(max_ratio = 0.001),  # No condition available for this check
        MixedNulls().add_condition_different_nulls_less_equal_to(), #add_condition_no_mixed_nulls(),  # Ensure no mixed null types
        MixedDataTypes().add_condition_rare_type_ratio_not_in_range(), #add_condition_no_mixed_types(),  # Ensure no mixed data types
        StringMismatch(similarity_threshold=params['similarity_threshold']).add_condition_number_variants_less_or_equal(5), #add_condition_number_of_mismatches_less_or_equal(5),  # Allow max 5 mismatches
        DataDuplicates().add_condition_ratio_less_or_equal(max_ratio = 0.05), #add_condition_no_duplicates(),  # Ensure no duplicate rows
        StringLengthOutOfBounds(),  # No condition available for this check
        ConflictingLabels().add_condition_ratio_of_conflicting_labels_less_or_equal(max_ratio=0), #.add_condition_no_conflicting_labels(),  # Ensure no conflicting labels
        OutlierSampleDetection().add_condition_outlier_ratio_less_or_equal(params['max_outliers_ratio'], params['outlier_score_threshold']),  # Outliers within limits
        FeatureLabelCorrelation().add_condition_feature_pps_less_than(threshold = 0.8), #.add_condition_feature_label_correlation_less_or_equal(0.9),  # Max correlation 0.9
        FeatureFeatureCorrelation(correlation_threshold=params['correlation_threshold']).add_condition_max_number_of_pairs_above_threshold(threshold = 0.9, n_pairs = 0), #.add_condition_feature_feature_correlation_less_or_equal(params['correlation_threshold']),  # Correlation within threshold
        IdentifierLabelCorrelation().add_condition_pps_less_or_equal(max_pps = 0) #add_condition_identifier_label_correlation_less_or_equal(0.8)  # Max identifier correlation 0.8
    ]
    
    # Allow user to select which checks to run
    print("Available checks:")
    for i, check in enumerate(checks):
        print(f"{i + 1}. {check.name}")
    selected_indices = input("Enter check numbers to run (comma-separated, e.g., 1,3,5): ")
    selected_indices = [int(idx.strip()) - 1 for idx in selected_indices.split(',') if idx.strip().isdigit()]
    
    selected_checks = [checks[i] for i in selected_indices]
    
    # Create and run suite
    suite = Suite("Custom Data Integrity Suite", *selected_checks)
    result = suite.run(dataset)
    
    # Show results
    result.show()
    return result

# Example usage
if __name__ == "__main__":
    import pandas as pd
    
    # Load or create a sample dataset
    data = pd.DataFrame({
        'feature1': [1, 1, 1, 1, 1],  # Single value column
        'feature2': ['@hello', 'world!', 'test123', 'data$', 'valid'],  # Special characters
        'feature3': [None, 'NULL', 'N/A', '', 'Missing'],  # Mixed nulls
        'feature4': [1, 100, 200, -50, 3000]  # Outliers
    })
    
    run_deepchecks(data)


deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 4 categorical features were inferred.: feature1, feature2, feature3, feature4


Available checks:
1. <bound method BaseCheck.name of <class 'deepchecks.tabular.checks.data_integrity.is_single_value.IsSingleValue'>>
2. <bound method BaseCheck.name of <class 'deepchecks.tabular.checks.data_integrity.special_chars.SpecialCharacters'>>
3. <bound method BaseCheck.name of <class 'deepchecks.tabular.checks.data_integrity.mixed_nulls.MixedNulls'>>
4. <bound method BaseCheck.name of <class 'deepchecks.tabular.checks.data_integrity.mixed_data_types.MixedDataTypes'>>
5. <bound method BaseCheck.name of <class 'deepchecks.tabular.checks.data_integrity.string_mismatch.StringMismatch'>>
6. <bound method BaseCheck.name of <class 'deepchecks.tabular.checks.data_integrity.data_duplicates.DataDuplicates'>>
7. <bound method BaseCheck.name of <class 'deepchecks.tabular.checks.data_integrity.string_length_out_of_bounds.StringLengthOutOfBounds'>>
8. <bound method BaseCheck.name of <class 'deepchecks.tabular.checks.data_integrity.conflicting_labels.ConflictingLabels'>>
9. <bound method B

Enter check numbers to run (comma-separated, e.g., 1,3,5):  1,2,3,6,10,11,12


Accordion(children=(VBox(children=(HTML(value='\n<h1 id="summary_47BJZPSH2RTN37TJUR3DKGUZ4">Custom Data Integr…

In [7]:
import deepchecks
from deepchecks.tabular import Suite
from deepchecks.tabular import Dataset
from deepchecks.tabular.checks import (
    IsSingleValue, SpecialCharacters, MixedNulls, MixedDataTypes, StringMismatch, DataDuplicates,
    StringLengthOutOfBounds, ConflictingLabels, OutlierSampleDetection, FeatureLabelCorrelation,
    FeatureFeatureCorrelation, IdentifierLabelCorrelation
)

# Define available checks (without parameters)
check_options = {
    "IsSingleValue": IsSingleValue(),
    "SpecialCharacters": SpecialCharacters(),
    "MixedNulls": MixedNulls(),
    "MixedDataTypes": MixedDataTypes(),
    "StringMismatch": StringMismatch(),  # Needs parameter
    "DataDuplicates": DataDuplicates(),
    "StringLengthOutOfBounds": StringLengthOutOfBounds(),
    "ConflictingLabels": ConflictingLabels(),
    "OutlierSampleDetection": OutlierSampleDetection(),  # Needs parameter
    "FeatureLabelCorrelation": FeatureLabelCorrelation(),
    "FeatureFeatureCorrelation": FeatureFeatureCorrelation(),  # Needs parameter
    "IdentifierLabelCorrelation": IdentifierLabelCorrelation()
}

# Checks that require user input parameters
checks_with_params = {
    "StringMismatch": ["similarity_threshold"],
    "OutlierSampleDetection": ["max_outliers_ratio", "outlier_score_threshold"],
    "FeatureFeatureCorrelation": ["correlation_threshold"]
}

def get_threshold_values(selected_tests):
    """Ask for threshold values only for selected tests that require them."""
    params = {}
    for test in selected_tests:
        if test in checks_with_params:
            for param in checks_with_params[test]:
                params[param] = float(input(f"Enter {param} for {test}: "))
    return params

def run_deepchecks(data1, data2, label=None):
    """Run selected deepchecks data integrity tests on the given dataset."""
    dataset1 = Dataset(data1, label=label)
    dataset2 = Dataset(data2, label=label)
    # Display available checks
    print("\nAvailable checks:")
    for i, test in enumerate(check_options.keys()):
        print(f"{i + 1}. {test}")

    # User selects which checks to run
    selected_indices = input("Enter check numbers to run (comma-separated, e.g., 1,3,5): ")
    selected_indices = [int(idx.strip()) - 1 for idx in selected_indices.split(',') if idx.strip().isdigit()]
    
    # Map indices to selected check names
    selected_tests = [list(check_options.keys())[i] for i in selected_indices]
    
    # Get threshold values only for selected tests that require them
    params = get_threshold_values(selected_tests)
    
    # Create selected checks with conditions where applicable
    selected_checks = []
    for test in selected_tests:
        if test == "StringMismatch":
            #check = StringMismatch(similarity_threshold=params['similarity_threshold']).add_condition_number_variants_less_or_equal(5)
            check = StringMismatch().add_condition_number_variants_less_or_equal(1)
        elif test == "OutlierSampleDetection":
            check = OutlierSampleDetection().add_condition_outlier_ratio_less_or_equal(
                params['max_outliers_ratio'], params['outlier_score_threshold']
            )
        elif test == "FeatureFeatureCorrelation":
            check = FeatureFeatureCorrelation(correlation_threshold=params['correlation_threshold']).add_condition_max_number_of_pairs_above_threshold(
                threshold=params['correlation_threshold'], n_pairs=0
            )
        elif test == "IsSingleValue":
            check = IsSingleValue().add_condition_not_single_value()
        elif test == "SpecialCharacters":
            check = SpecialCharacters().add_condition_ratio_of_special_characters_less_or_equal(max_ratio = 0.001)  # No condition available for this check   
        elif test == "MixedNulls":
            check = MixedNulls().add_condition_different_nulls_less_equal_to()
        elif test == "MixedDataTypes":
            check = MixedDataTypes().add_condition_rare_type_ratio_not_in_range((0.01, 0.1))
        elif test == "DataDuplicates":
            check = DataDuplicates().add_condition_ratio_less_or_equal(max_ratio=0.05)
        elif test == "ConflictingLabels":
            check = ConflictingLabels().add_condition_ratio_of_conflicting_labels_less_or_equal(max_ratio=0)
        elif test == "FeatureLabelCorrelation":
            check = FeatureLabelCorrelation().add_condition_feature_pps_less_than(threshold=0.8)
        elif test == "IdentifierLabelCorrelation":
            check = IdentifierLabelCorrelation().add_condition_pps_less_or_equal(max_pps=0)
        else:
            check = check_options[test]

        selected_checks.append(check)
    
    # Create and run suite
    suite = Suite("Custom Data Integrity Suite", *selected_checks)
    result = suite.run(dataset1, dataset2)

    # Show results
    result.show()
    return result

# Example usage
if __name__ == "__main__":
    import pandas as pd
    
    # Load or create a sample dataset
    data = pd.DataFrame({
        'feature1': [1, 1, 1, 1, 1],  # Single value column
        'feature2': ['@hello', 'world!', 'test123', 'data$', 'valid'],  # Special characters
        'feature3': [None, 'NULL', 'N/A', '', 'Missing'],  # Mixed nulls
        'feature4': [1, 100, 200, -50, 3000]  # Outliers
    })
    
    run_deepchecks(data, data)


deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 4 categorical features were inferred.: feature1, feature2, feature3, feature4
deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 4 categorical features were inferred.: feature1, feature2, feature3, feature4



Available checks:
1. IsSingleValue
2. SpecialCharacters
3. MixedNulls
4. MixedDataTypes
5. StringMismatch
6. DataDuplicates
7. StringLengthOutOfBounds
8. ConflictingLabels
9. OutlierSampleDetection
10. FeatureLabelCorrelation
11. FeatureFeatureCorrelation
12. IdentifierLabelCorrelation


KeyboardInterrupt: Interrupted by user

Enter check numbers to run (comma-separated, e.g., 1,3,5):  1


In [42]:
data = pd.DataFrame({
        'feature1': [1, 1, 1, 1, 1],  # Single value column
        'feature2': ['@hello', 'world!', 'test123', 'data$', 'valid'],  # Special characters
        'feature3': [None, 'NULL', 'N/A', '', 'Missing'],  # Mixed nulls
        'feature4': [1, 100, 200, -50, 3000]  # Outliers
    })

from deepchecks.tabular.suites import data_integrity
suite = data_integrity()
result = suite.run(train_dataset = data, test_dataset = data)
result.show()

deepchecks - WARNING - Received a "pandas.DataFrame" instance. It is recommended to pass a "deepchecks.tabular.Dataset" instance by initializing it with the data and metadata, for example by doing "Dataset(dataframe, label=label, cat_features=cat_features)"
deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 4 categorical features were inferred.: feature1, feature2, feature3, feature4
deepchecks - WARNING - Received a "pandas.DataFrame" instance. It is recommended to pass a "deepchecks.tabular.Dataset" instance by initializing it with the data and metadata, for example by doing "Dataset(dataframe, label=label, cat_features=cat_features)"
deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorica

Accordion(children=(VBox(children=(HTML(value='\n<h1 id="summary_7113MY6JJE48JUYJ5EBI12N59">Data Integrity Sui…

In [1]:
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix

In [5]:
# Import Required Libraries
import deepchecks
from deepchecks.tabular.suites import data_integrity, train_test_validation, model_evaluation
from ipywidgets import interactive, VBox, HBox, Dropdown, FloatSlider, Button, Checkbox, Output, Label
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from docx import Document  # For saving the results to a Word document
from datetime import datetime  # To generate unique filenames

# -------------------- Step 1: Create Sample Data (Optional) --------------------
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=1000, n_features=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -------------------- Step 2: Train a Simple Model (Optional) --------------------
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# -------------------- Step 3: Create Jupyter Widgets --------------------
available_suites = {
    'Data Integrity': data_integrity(),
    'Train-Test Validation': train_test_validation(),
    'Model Evaluation': model_evaluation()
}

# Dropdown to select the suite
suite_dropdown = Dropdown(
    options=list(available_suites.keys()),
    value='Data Integrity',
    description='Suite:'
)

# Checkboxes and sliders for each test (Populated dynamically)
test_checkboxes = {}
threshold_sliders = {}
test_selection_box = VBox([])  # This will hold both checkboxes and sliders for each test

# Button to run the suite
run_button = Button(description="Run Suite")

# Output to display the suite results
output = Output()

# -------------------- Step 4: Update Test Selection When Suite Changes --------------------
def update_test_selection(change):
    """Update the available tests (checkboxes and sliders) when the suite changes."""
    global test_checkboxes, threshold_sliders
    selected_suite = change['new']
    
    # Clear old checkboxes and sliders
    test_checkboxes.clear()
    threshold_sliders.clear()
    
    # Create new checkboxes and sliders for each test in the selected suite
    test_widgets = []
    for test_key, test_obj in available_suites[selected_suite].checks.items():
        # Create a checkbox for the test
        test_checkboxes[test_obj.name()] = Checkbox(
            value=True, 
            description=test_obj.name()
        )
        
        # Create a slider for the threshold of this test
        threshold_sliders[test_obj.name()] = FloatSlider(
            value=0.8, 
            min=0.0, 
            max=1.0, 
            step=0.05, 
            description='Threshold'
        )
        
        # Add the checkbox and slider side-by-side (HBox)
        test_widgets.append(HBox([test_checkboxes[test_obj.name()], threshold_sliders[test_obj.name()]]))
    
    # Update the test selection box with all the new checkboxes and sliders
    test_selection_box.children = test_widgets

# Call this function once at the start to initialize the UI
update_test_selection({'new': 'Data Integrity'})

# Attach the function to observe changes in the dropdown selection
suite_dropdown.observe(update_test_selection, names='value')

# -------------------- Step 5: Run the Selected Suite and Save Results --------------------
def run_selected_suite(b):
    """Run the selected suite with selected tests and thresholds, then save the results to a Word file."""
    with output:
        output.clear_output()
        
        selected_suite_name = suite_dropdown.value
        selected_suite = available_suites[selected_suite_name]
        
        # Filter the selected tests
        selected_tests = [test for test in selected_suite.checks if test_checkboxes[test[1].name()].value]
        
        # Set the custom threshold for each selected test
        for test in selected_tests:
            test_name = test[1].name()
            print("Set the custom threshold for each selected test ", test_name)
            threshold_value = threshold_sliders[test_name].value
            test.add_condition_test_score_greater_than(threshold_value)
        
        # Run the selected suite
        if selected_suite_name == 'Data Integrity':
            result = selected_suite.run(X_train)
        elif selected_suite_name == 'Train-Test Validation':
            result = selected_suite.run(X_train, X_test, y_train, y_test)
        elif selected_suite_name == 'Model Evaluation':
            result = selected_suite.run(X_train, y_train, X_test, y_test, model)
        
        result.show()
        
        # -------------------- Save the Results to a Word File --------------------
        try:
            # Create a new Word document
            doc = Document()
            timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
            
            doc.add_heading(f'Deepchecks Report - {selected_suite_name}', level=1)
            doc.add_paragraph(f'Run Timestamp: {timestamp}\n')
            
            summary = result.to_dataframe()
            doc.add_heading('Test Summary', level=2)
            
            for index, row in summary.iterrows():
                doc.add_paragraph(f'Test: {row["Check"]} | Status: {row["Result"]} | Value: {row["Value"]}')
            
            doc.add_heading('Detailed Results', level=2)
            for check_result in result.get_not_passed_checks():
                doc.add_paragraph(f'Check Name: {check_result.name}')
                doc.add_paragraph(f'Description: {check_result.description}')
                doc.add_paragraph(f'Result: {check_result.result}')
                doc.add_paragraph(f'Message: {check_result.display()}')
                doc.add_paragraph('---' * 10)
            
            file_name = f'{selected_suite_name}_Report_{timestamp}.docx'
            doc.save(file_name)
            output.append_stdout(f"Results saved to {file_name}\n")
        except Exception as e:
            output.append_stdout(f"Error saving results: {str(e)}\n")

# Attach the "run suite" button to the handler function
run_button.on_click(run_selected_suite)

# -------------------- Step 6: Display the Jupyter Widget UI --------------------
# Layout for the widget
ui = VBox([
    suite_dropdown,  # Dropdown to select the suite
    test_selection_box,  # Box containing test checkboxes and sliders
    run_button,  # Button to run the suite
    output  # Output display for test results and file save status
])

# Display the interactive UI
display(ui)


In [6]:
# Import Required Libraries
import deepchecks
from deepchecks.tabular.suites import data_integrity, train_test_validation, model_evaluation
from ipywidgets import VBox, HBox, Dropdown, FloatSlider, Button, Checkbox, Output
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from docx import Document  # For saving results
from datetime import datetime  # For unique filenames

# -------------------- Step 1: Create Sample Data --------------------
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=1000, n_features=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


from deepchecks.tabular import Dataset

# Convert NumPy arrays to Deepchecks Dataset
train_dataset = Dataset(pd.DataFrame(X_train), label=pd.Series(y_train))
test_dataset = Dataset(pd.DataFrame(X_test), label=pd.Series(y_test))


# -------------------- Step 2: Train a Simple Model --------------------
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# -------------------- Step 3: Create Jupyter Widgets --------------------
available_suites = {
    'Data Integrity': data_integrity(),
    'Train-Test Validation': train_test_validation(),
    'Model Evaluation': model_evaluation()
}

# Dropdown to select the suite
suite_dropdown = Dropdown(
    options=list(available_suites.keys()),
    value='Data Integrity',
    description='Suite:'
)

# Checkboxes and sliders for each test (Populated dynamically)
test_checkboxes = {}
threshold_sliders = {}
test_selection_box = VBox([])

# Button to run the suite
run_button = Button(description="Run Suite")

# Output to display results
output = Output()

# -------------------- Step 4: Update Test Selection When Suite Changes --------------------
def update_test_selection(change):
    """Update available tests (checkboxes and sliders) when the suite changes."""
    global test_checkboxes, threshold_sliders
    selected_suite = change['new']
    
    # Clear old checkboxes and sliders
    test_checkboxes.clear()
    threshold_sliders.clear()
    
    test_widgets = []
    for test_key, test_obj in available_suites[selected_suite].checks.items():
        test_name = test_obj.name()
        
        # Create a checkbox for the test
        test_checkboxes[test_name] = Checkbox(value=True, description=test_name)
        
        # Add threshold sliders only for relevant tests
        slider_widget = None
        if test_name == "Outlier Sample Detection":
            # Needs two sliders for max_outliers_ratio and outlier_score_threshold
            max_outliers_slider = FloatSlider(value=0.01, min=0.0, max=0.1, step=0.001, description='Max Outliers')
            score_threshold_slider = FloatSlider(value=0.9, min=0.0, max=1.0, step=0.05, description='Score Threshold')
            threshold_sliders[test_name] = (max_outliers_slider, score_threshold_slider)
            slider_widget = HBox([max_outliers_slider, score_threshold_slider])
        
        elif hasattr(test_obj, "add_condition_greater_than"):
            # Generic case for tests that support 'add_condition_greater_than'
            threshold_sliders[test_name] = FloatSlider(value=0.8, min=0.0, max=1.0, step=0.05, description='Threshold')
            slider_widget = threshold_sliders[test_name]

        row_widgets = [test_checkboxes[test_name]]
        if slider_widget:
            row_widgets.append(slider_widget)
        
        test_widgets.append(HBox(row_widgets))
    
    test_selection_box.children = test_widgets

update_test_selection({'new': 'Data Integrity'})  # Initialize UI
suite_dropdown.observe(update_test_selection, names='value')

# -------------------- Step 5: Run the Selected Suite and Save Results --------------------
def run_selected_suite(b):
    """Run the selected suite with thresholds and save results to a Word file."""
    with output:
        output.clear_output()
        
        selected_suite_name = suite_dropdown.value
        selected_suite = available_suites[selected_suite_name]
        
        # Filter selected tests
        selected_tests = [test for test in selected_suite.checks.items() if test_checkboxes[test[1].name()].value]
        
        # Set custom thresholds for selected tests
        for test in selected_tests:
            test_name = test[1].name()
            try:
                if test_name == "Outlier Sample Detection":
                    # Apply two threshold values
                    max_outliers_ratio = threshold_sliders[test_name][0].value
                    outlier_score_threshold = threshold_sliders[test_name][1].value
                    test[1].add_condition_outlier_ratio_less_or_equal(
                        max_outliers_ratio=max_outliers_ratio,
                        outlier_score_threshold=outlier_score_threshold
                    )
                elif test_name in threshold_sliders:
                    # Apply single threshold value
                    threshold_value = threshold_sliders[test_name].value
                    test[1].add_condition_greater_than(threshold_value)
            except AttributeError:
                output.append_stdout(f"Warning: No threshold condition for {test_name}\n")
        
        # Run the selected suite with the correct dataset format
        if selected_suite_name == 'Data Integrity':
            result = selected_suite.run(train_dataset)
        elif selected_suite_name == 'Train-Test Validation':
            result = selected_suite.run(train_dataset, test_dataset)
        elif selected_suite_name == 'Model Evaluation':
            result = selected_suite.run(train_dataset, test_dataset, model)
        
        # Ensure all tab titles are strings before displaying
        for check_result in result.results:
            check_result = [str(item) if isinstance(item, int) else item for item in check_result.display]

        result.show()
        
        # -------------------- Save the Results to a Word File --------------------
        try:
            doc = Document()
            timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
            
            doc.add_heading(f'Deepchecks Report - {selected_suite_name}', level=1)
            doc.add_paragraph(f'Run Timestamp: {timestamp}\n')
            
            summary = result.to_dataframe()
            doc.add_heading('Test Summary', level=2)
            
            for index, row in summary.iterrows():
                doc.add_paragraph(f'Test: {row["Check"]} | Status: {row["Result"]} | Value: {row["Value"]}')
            
            doc.add_heading('Detailed Results', level=2)
            for check_result in result.get_not_passed_checks():
                doc.add_paragraph(f'Check Name: {check_result.name}')
                doc.add_paragraph(f'Description: {check_result.description}')
                doc.add_paragraph(f'Result: {check_result.result}')
                doc.add_paragraph(f'Message: {check_result.display()}')
                doc.add_paragraph('---' * 10)
            
            file_name = f'{selected_suite_name}_Report_{timestamp}.docx'
            doc.save(file_name)
            output.append_stdout(f"Results saved to {file_name}\n")
        except Exception as e:
            output.append_stdout(f"Error saving results: {str(e)}\n")

# Attach button click event
run_button.on_click(run_selected_suite)

# -------------------- Step 6: Display the Jupyter Widget UI --------------------
ui = VBox([
    suite_dropdown, 
    test_selection_box,  
    run_button,  
    output  
])

display(ui)


deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 0 categorical features were inferred.
deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 0 categorical features were inferred.


In [7]:
### WORKING CODE FILE


import deepchecks

# Check if CheckFailure is part of the deepchecks suite (check if it's defined in your version)
CheckFailure = deepchecks.CheckFailure

# Import Required Libraries
import deepchecks
from deepchecks.tabular.suites import data_integrity, train_test_validation, model_evaluation
from ipywidgets import VBox, HBox, Dropdown, FloatSlider, Button, Checkbox, Output
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from docx import Document  # For saving results
from datetime import datetime  # For unique filenames

# -------------------- Step 1: Create Sample Data --------------------
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=1000, n_features=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


from deepchecks.tabular import Dataset

# Convert NumPy arrays to Deepchecks Dataset
train_dataset = Dataset(pd.DataFrame(X_train), label=pd.Series(y_train))
test_dataset = Dataset(pd.DataFrame(X_test), label=pd.Series(y_test))


# -------------------- Step 2: Train a Simple Model --------------------
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# -------------------- Step 3: Create Jupyter Widgets --------------------
available_suites = {
    'Data Integrity': data_integrity(),
    'Train-Test Validation': train_test_validation(),
    'Model Evaluation': model_evaluation()
}

# Dropdown to select the suite
suite_dropdown = Dropdown(
    options=list(available_suites.keys()),
    value='Data Integrity',
    description='Suite:'
)

# Checkboxes and sliders for each test (Populated dynamically)
test_checkboxes = {}
threshold_sliders = {}
test_selection_box = VBox([])

# Button to run the suite
run_button = Button(description="Run Suite")

# Output to display results
output = Output()

# -------------------- Step 4: Update Test Selection When Suite Changes --------------------
def update_test_selection(change):
    """Update available tests (checkboxes and sliders) when the suite changes."""
    global test_checkboxes, threshold_sliders
    selected_suite = change['new']
    
    # Clear old checkboxes and sliders
    test_checkboxes.clear()
    threshold_sliders.clear()
    
    test_widgets = []
    for test_key, test_obj in available_suites[selected_suite].checks.items():
        test_name = test_obj.name()
        
        # Create a checkbox for the test
        test_checkboxes[test_name] = Checkbox(value=True, description=test_name)
        
        # Add threshold sliders only for relevant tests
        slider_widget = None
        if test_name == "Outlier Sample Detection":
            # Needs two sliders for max_outliers_ratio and outlier_score_threshold
            max_outliers_slider = FloatSlider(value=0.01, min=0.0, max=0.1, step=0.001, description='Max Outliers')
            score_threshold_slider = FloatSlider(value=0.9, min=0.0, max=1.0, step=0.05, description='Score Threshold')
            threshold_sliders[test_name] = (max_outliers_slider, score_threshold_slider)
            slider_widget = HBox([max_outliers_slider, score_threshold_slider])
        
        elif hasattr(test_obj, "add_condition_greater_than"):
            # Generic case for tests that support 'add_condition_greater_than'
            threshold_sliders[test_name] = FloatSlider(value=0.8, min=0.0, max=1.0, step=0.05, description='Threshold')
            slider_widget = threshold_sliders[test_name]

        row_widgets = [test_checkboxes[test_name]]
        if slider_widget:
            row_widgets.append(slider_widget)
        
        test_widgets.append(HBox(row_widgets))
    
    test_selection_box.children = test_widgets

update_test_selection({'new': 'Data Integrity'})  # Initialize UI
suite_dropdown.observe(update_test_selection, names='value')

# -------------------- Step 5: Run the Selected Suite and Save Results --------------------
def run_selected_suite(b):
    """Run the selected suite with thresholds and save results to a Word file."""
    with output:
        output.clear_output()
        
        selected_suite_name = suite_dropdown.value
        selected_suite = available_suites[selected_suite_name]
        
        # Filter selected tests
        selected_tests = [test for test in selected_suite.checks.items() if test_checkboxes[test[1].name()].value]
        
        # Set custom thresholds for selected tests
        for test in selected_tests:
            test_name = test[1].name()
            try:
                if test_name == "Outlier Sample Detection":
                    # Apply two threshold values
                    max_outliers_ratio = threshold_sliders[test_name][0].value
                    outlier_score_threshold = threshold_sliders[test_name][1].value
                    test[1].add_condition_outlier_ratio_less_or_equal(
                        max_outliers_ratio=max_outliers_ratio,
                        outlier_score_threshold=outlier_score_threshold
                    )
                elif test_name in threshold_sliders:
                    # Apply single threshold value
                    threshold_value = threshold_sliders[test_name].value
                    test[1].add_condition_greater_than(threshold_value)
            except AttributeError:
                output.append_stdout(f"Warning: No threshold condition for {test_name}\n")
        
        # Run the selected suite with the correct dataset format
        if selected_suite_name == 'Data Integrity':
            result = selected_suite.run(train_dataset)
        elif selected_suite_name == 'Train-Test Validation':
            result = selected_suite.run(train_dataset, test_dataset)
        elif selected_suite_name == 'Model Evaluation':
            result = selected_suite.run(train_dataset, test_dataset, model)
        
        # Now show the results after processing
        #result.show()
        result.save_as_html('my_results.html')
        
        # -------------------- Save the Results to a Word File --------------------
        try:
            doc = Document()
            timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
            
            doc.add_heading(f'Deepchecks Report - {selected_suite_name}', level=1)
            doc.add_paragraph(f'Run Timestamp: {timestamp}\n')
            
            summary = result.to_dataframe()
            doc.add_heading('Test Summary', level=2)
            
            for index, row in summary.iterrows():
                doc.add_paragraph(f'Test: {row["Check"]} | Status: {row["Result"]} | Value: {row["Value"]}')
            
            doc.add_heading('Detailed Results', level=2)
            for check_result in result.get_not_passed_checks():
                doc.add_paragraph(f'Check Name: {check_result.name}')
                doc.add_paragraph(f'Description: {check_result.description}')
                doc.add_paragraph(f'Result: {check_result.result}')
                doc.add_paragraph(f'Message: {check_result.display()}')
                doc.add_paragraph('---' * 10)
            
            file_name = f'{selected_suite_name}_Report_{timestamp}.docx'
            print(file_name)
            doc.save(file_name)
            output.append_stdout(f"Results saved to {file_name}\n")
        except Exception as e:
            output.append_stdout(f"Error saving results: {str(e)}\n")

# Attach button click event
run_button.on_click(run_selected_suite)

# -------------------- Step 6: Display the Jupyter Widget UI --------------------
ui = VBox([
    suite_dropdown, 
    test_selection_box,  
    run_button,  
    output  
])

display(ui)


deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 0 categorical features were inferred.
deepchecks - WARNING - It is recommended to initialize Dataset with categorical features by doing "Dataset(df, cat_features=categorical_list)". No categorical features were passed, therefore heuristically inferring categorical features in the data. 0 categorical features were inferred.


In [ ]:
#!conda install -c conda-forge deepchecks

In [ ]:
#import sys
#!{sys.executable} -m pip install deepchecks --quiet --upgrade

In [ ]:
import deepchecks
from deepchecks.tabular.suites import data_integrity

#from deepchecks.tabular.suites import data_integrity, train_test_validation, model_evaluation

In [ ]:
import docx

In [8]:

# Import Required Libraries
import deepchecks
from deepchecks.suites import data_integrity, train_test_validation, model_evaluation
from ipywidgets import interactive, VBox, HBox, Dropdown, FloatSlider, Button, Checkbox, Output
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from docx import Document  # For saving the results to a Word document
from datetime import datetime  # To generate unique filenames

# -------------------- Step 1: Create Sample Data (Optional) --------------------
# If you have your own data, you can skip this step and load your dataset instead.
from sklearn.datasets import make_classification

X, y = make_classification(n_samples=1000, n_features=10, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -------------------- Step 2: Train a Simple Model (Optional) --------------------
# Train a RandomForest model as an example
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# -------------------- Step 3: Create Jupyter Widgets --------------------
# Available deepchecks suites
available_suites = {
    'Data Integrity': data_integrity(),
    'Train-Test Validation': train_test_validation(),
    'Model Evaluation': model_evaluation()
}


# Dropdown widget to select the suite
suite_dropdown = Dropdown(
    options=list(available_suites.keys()),
    value='Data Integrity',
    description='Suite:'
)

# Checkboxes to select which tests to run within the suite
test_checkboxes = {test[0]: Checkbox(value=True, description=test[1].name()) 
                   for test in available_suites['Data Integrity'].checks.items()}
test_selection_box = VBox(list(test_checkboxes.values()))

# Slider to set the threshold for passing the test
threshold_slider = FloatSlider(
    value=0.8, 
    min=0.0, 
    max=1.0, 
    step=0.05, 
    description='Threshold:'
)

# Button to run the suite
run_button = Button(description="Run Suite")

# Output area to display the suite results
output = Output()

# -------------------- Step 4: Update Test Selection When Suite Changes --------------------
def update_test_selection(change):
    """Update the available tests when the suite changes."""
    global test_checkboxes
    selected_suite = change['new']
    test_checkboxes = {test[0]: Checkbox(value=True, description=test[1].name()) 
                       for test in available_suites[selected_suite].checks.items()}
    test_selection_box.children = list(test_checkboxes.values())

suite_dropdown.observe(update_test_selection, names='value')

# -------------------- Step 5: Run the Selected Suite and Save Results --------------------
def run_selected_suite(b):
    """Run the selected suite with selected tests and thresholds, then save the results to a Word file."""
    with output:
        output.clear_output()
        selected_suite_name = suite_dropdown.value
        selected_suite = available_suites[selected_suite_name]
        print(selected_suite)
        
        #for test in selected_suite.checks:
            #print(test[1])
            #print(test)
            #print(test_checkboxes[0].value)
        # Filter the selected tests
        selected_tests = [test for i, test in selected_suite.checks.items() if test_checkboxes[i].value]
        print("selected_tests")
        print(selected_tests)
        # Set a threshold for each test
        for test in selected_tests:
            print(test)
            test.add_condition_test_score_greater_than(threshold_slider.value)
        
        # Run the suite and display the results
        if selected_suite_name == 'Data Integrity':
            result = selected_suite.run(X_train)
        elif selected_suite_name == 'Train-Test Validation':
            result = selected_suite.run(X_train, X_test, y_train, y_test)
        elif selected_suite_name == 'Model Evaluation':
            result = selected_suite.run(X_train, y_train, X_test, y_test, model)
        
        result.show()
        
        # -------------------- Save the Results to a Word File --------------------
        try:
            # Create a new Word document
            doc = Document()
            
            # Add a title with the suite name and timestamp
            timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
            doc.add_heading(f'Deepchecks Report - {selected_suite_name}', level=1)
            doc.add_paragraph(f'Run Timestamp: {timestamp}\n')
            
            # Add a summary of the suite results
            summary = result.to_dataframe()
            doc.add_heading('Test Summary', level=2)
            
            for index, row in summary.iterrows():
                doc.add_paragraph(f'Test: {row["Check"]} | Status: {row["Result"]} | Value: {row["Value"]}')
            
            # Add detailed results for each test
            doc.add_heading('Detailed Results', level=2)
            for check_result in result.get_not_passed_checks():
                doc.add_paragraph(f'Check Name: {check_result.name}')
                doc.add_paragraph(f'Description: {check_result.description}')
                doc.add_paragraph(f'Result: {check_result.result}')
                doc.add_paragraph(f'Message: {check_result.display()}')
                doc.add_paragraph('---' * 10)  # Divider
            
            # Save the Word document
            file_name = f'{selected_suite_name}_Report_{timestamp}.docx'
            doc.save(file_name)
            
            output.append_stdout(f"Results saved to {file_name}\n")
        except Exception as e:
            output.append_stdout(f"Error saving results: {str(e)}\n")

# Attach the "run suite" button to the handler function
run_button.on_click(run_selected_suite)

# -------------------- Step 6: Display the Jupyter Widget UI --------------------
# Layout for the widget
ui = VBox([
    suite_dropdown,  # Dropdown to select the suite
    test_selection_box,  # Checkboxes to select which tests to run
    threshold_slider,  # Slider to set test pass threshold
    run_button,  # Button to run the suite
    output  # Output display for test results and file save status
])

# Display the interactive UI
display(ui)

C:\Users\DELL\AppData\Local\Programs\Python\Python39\lib\site-packages\deepchecks\suites.py:21: DeprecationWarning:

Ability to import tabular suites from the `deepchecks.suites` is deprecated, please import from `deepchecks.tabular.suites` instead



In [9]:
from deepchecks.tabular.suites import data_integrity
suite = data_integrity(columns=['a', 'b', 'c'], n_samples=1_000_000)
result = suite.run()

DeepchecksValueError: At least one dataset (or model) must be passed to the method!

In [10]:
for i,j in data_integrity().checks.items():
    print(j.)

SyntaxError: invalid syntax (3432277579.py, line 2)

In [11]:
print((j.params().keys())[0])
j.params()


NameError: name 'j' is not defined

In [12]:
for i,j in data_integrity().checks.items():
    print(i, j.clean_conditions(), j)

0 None IsSingleValue
1 None SpecialCharacters
2 None MixedNulls
3 None MixedDataTypes
4 None StringMismatch
5 None DataDuplicates
6 None StringLengthOutOfBounds
7 None ConflictingLabels
8 None OutlierSampleDetection
9 None FeatureLabelCorrelation(ppscore_params={}, random_state=42)
10 None FeatureFeatureCorrelation
11 None IdentifierLabelCorrelation(ppscore_params={})


In [13]:
test_selection_box

In [14]:
# Slider to set the threshold for passing the test
threshold_slider = FloatSlider(
    value=0.8, 
    min=0.0, 
    max=1.0, 
    step=0.05, 
    description='Threshold:'
)


In [15]:
threshold_slider

FloatSlider(value=0.8, description='Threshold:', max=1.0, step=0.05)

In [17]:
run_button

Button(description='Run Suite', style=ButtonStyle())

In [18]:
# Import Required Libraries
import deepchecks
from deepchecks.tabular.suites import data_integrity, train_test_validation, model_evaluation

# Available deepchecks suites
available_suites = {
    'Data Integrity': data_integrity(),
    'Train-Test Validation': train_test_validation(),
    'Model Evaluation': model_evaluation()
}

In [19]:
for test in available_suites['Data Integrity'].checks.items():
    #print(test)
    print(test[1].name())
    #print()
    print(test[1].config())
    print(test[1].metadata())



Is Single Value
{'class_name': 'IsSingleValue', 'module_name': 'deepchecks.tabular.checks.data_integrity.is_single_value', 'params': {'columns': None, 'ignore_columns': None, 'ignore_nan': True, 'n_samples': 10000000, 'random_state': 42}, 'version': '0.19.0'}
{'name': 'Is Single Value', 'params': {'columns': None, 'ignore_columns': None, 'ignore_nan': True, 'n_samples': 10000000, 'random_state': 42}, 'summary': 'Check if there are columns which have only a single unique value in all rows.'}
Special Characters
{'class_name': 'SpecialCharacters', 'module_name': 'deepchecks.tabular.checks.data_integrity.special_chars', 'params': {'columns': None, 'ignore_columns': None, 'n_most_common': 2, 'n_top_columns': 10, 'n_samples': 10000000, 'random_state': 42}, 'version': '0.19.0'}
{'name': 'Special Characters', 'params': {'columns': None, 'ignore_columns': None, 'n_most_common': 2, 'n_top_columns': 10, 'n_samples': 10000000, 'random_state': 42}, 'summary': 'Search in column[s] for values that co

In [20]:
test[1].conditions_decision()

TypeError: conditions_decision() missing 1 required positional argument: 'result'

In [21]:
#print(test[1].metadata())

print(test[1].params())
print(test[1].name())
#test[1].remove_condition(0)

from deepchecks.tabular.checks import IdentifierLabelCorrelation

test[1].ppscore_params={0.1}
test[1].metadata()

{'ppscore_params': {}}
Identifier Label Correlation


{'name': 'Identifier Label Correlation',
 'params': {'ppscore_params': {0.1}, 'n_samples': 1000000, 'random_state': 42},
 'summary': 'Check if identifiers (Index/Date) can be used to predict the label.'}

In [22]:
from deepchecks.tabular.suites import data_integrity
from deepchecks.checks import MixedNulls  # Ensure this import is correct

suite = data_integrity()

# Step 2: Print check types to confirm the type of each check
for check in suite.checks.items():
    #print(check[1])
    print("1111---")
    print(check[1])
    print("2222---")
    print(check[1].metadata())
    print("3333---")
    print(check[1].name())

1111---
IsSingleValue
	Conditions:
		0: Does not contain only a single value
2222---
{'name': 'Is Single Value', 'params': {'columns': None, 'ignore_columns': None, 'ignore_nan': True, 'n_samples': 10000000, 'random_state': 42}, 'summary': 'Check if there are columns which have only a single unique value in all rows.'}
3333---
Is Single Value
1111---
SpecialCharacters
	Conditions:
		0: Ratio of samples containing solely special character is less or equal to 0.1%
2222---
{'name': 'Special Characters', 'params': {'columns': None, 'ignore_columns': None, 'n_most_common': 2, 'n_top_columns': 10, 'n_samples': 10000000, 'random_state': 42}, 'summary': 'Search in column[s] for values that contains only special characters.'}
3333---
Special Characters
1111---
MixedNulls
	Conditions:
		0: Number of different null types is less or equal to 1
2222---
{'name': 'Mixed Nulls', 'params': {'null_string_list': None, 'check_nan': True, 'columns': None, 'ignore_columns': None, 'n_top_columns': 10, 'aggre

C:\Users\DELL\AppData\Local\Programs\Python\Python39\lib\site-packages\deepchecks\checks.py:21: DeprecationWarning:

Ability to import tabular checks from the `deepchecks.checks` is deprecated, please import from `deepchecks.tabular.checks` instead



In [23]:
print("1111---")
print(check[1])
print("2222---")
print(check[1].metadata())
print("3333---")
print(check[1].name())

suite.checks.values()[1]


1111---
IdentifierLabelCorrelation(ppscore_params={})
	Conditions:
		0: Identifier columns PPS is less or equal to 0
2222---
{'name': 'Identifier Label Correlation', 'params': {'ppscore_params': {}, 'n_samples': 1000000, 'random_state': 42}, 'summary': 'Check if identifiers (Index/Date) can be used to predict the label.'}
3333---
Identifier Label Correlation


TypeError: 'odict_values' object is not subscriptable

In [24]:
from deepchecks.tabular.suites import data_integrity
from deepchecks.checks import MixedNulls  # Ensure this import is correct

suite = data_integrity()

# Step 2: Print check types to confirm the type of each check
for check in suite.checks.items():
    print(check[1].)

# Step 3: Access the MixedNulls check correctly
for check in suite.checks:
    if isinstance(check, MixedNulls):  # Ensure the name of the class is correct
        mixed_nulls_check = check
        break
else:
    raise ValueError("MixedNulls check not found in the Data Integrity suite.")

# Step 4: Update the condition
mixed_nulls_check.conditions = {}  # Clears all existing conditions
mixed_nulls_check.add_condition('proportion of mixed nulls is less than', threshold=0.1)


SyntaxError: invalid syntax (1490723542.py, line 8)

In [25]:
print(check.)

SyntaxError: invalid syntax (1969544045.py, line 1)

In [26]:
from deepchecks.tabular.suites import data_integrity
from deepchecks.checks import MixedNulls  # Ensure this import is correct

# Step 1: Create the Data Integrity Suite
suite = data_integrity()

# Step 2: Convert the odict_values to a list
checks_list = list(suite.checks.items())  # Convert to list to make it subscriptable
#print(checks_list)
print(checks_list[0][1])
print(dir(checks_list[0][1]))

checks_list[0][1].config()

IsSingleValue
	Conditions:
		0: Does not contain only a single value
['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_conditions', '_conditions_index', '_prepare_config', 'add_condition', 'add_condition_not_single_value', 'clean_conditions', 'columns', 'conditions_decision', 'config', 'context_type', 'from_config', 'from_json', 'ignore_columns', 'ignore_nan', 'metadata', 'n_samples', 'name', 'params', 'random_state', 'remove_condition', 'run', 'run_logic', 'to_json']


{'class_name': 'IsSingleValue',
 'module_name': 'deepchecks.tabular.checks.data_integrity.is_single_value',
 'params': {'columns': None,
  'ignore_columns': None,
  'ignore_nan': True,
  'n_samples': 10000000,
  'random_state': 42},
 'version': '0.19.0'}

In [27]:
# Output area to display the suite results
output = Output()

# -------------------- Step 4: Update Test Selection When Suite Changes --------------------
def update_test_selection(change):
    """Update the available tests when the suite changes."""
    global test_checkboxes
    selected_suite = change['new']
    test_checkboxes = {test[0]: Checkbox(value=True, description=test[1].name()) 
                       for test in available_suites[selected_suite].checks.items()}
    test_selection_box.children = list(test_checkboxes.values())

suite_dropdown.observe(update_test_selection, names='value')

# -------------------- Step 5: Run the Selected Suite and Save Results --------------------
def run_selected_suite(b):
    """Run the selected suite with selected tests and thresholds, then save the results to a Word file."""
    with output:
        output.clear_output()
        selected_suite_name = suite_dropdown.value
        selected_suite = available_suites[selected_suite_name]
        
        # Filter the selected tests
        selected_tests = [test for test in selected_suite.checks if test_checkboxes[test.name].value]
        
        # Set a threshold for each test
        for test in selected_tests:
            test.add_condition_test_score_greater_than(threshold_slider.value)
        
        # Run the suite and display the results
        if selected_suite_name == 'Data Integrity':
            result = selected_suite.run(X_train)
        elif selected_suite_name == 'Train-Test Validation':
            result = selected_suite.run(X_train, X_test, y_train, y_test)
        elif selected_suite_name == 'Model Evaluation':
            result = selected_suite.run(X_train, y_train, X_test, y_test, model)
        
        result.show()
        
        # -------------------- Save the Results to a Word File --------------------
        try:
            # Create a new Word document
            doc = Document()
            
            # Add a title with the suite name and timestamp
            timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
            doc.add_heading(f'Deepchecks Report - {selected_suite_name}', level=1)
            doc.add_paragraph(f'Run Timestamp: {timestamp}\n')
            
            # Add a summary of the suite results
            summary = result.to_dataframe()
            doc.add_heading('Test Summary', level=2)
            
            for index, row in summary.iterrows():
                doc.add_paragraph(f'Test: {row["Check"]} | Status: {row["Result"]} | Value: {row["Value"]}')
            
            # Add detailed results for each test
            doc.add_heading('Detailed Results', level=2)
            for check_result in result.get_not_passed_checks():
                doc.add_paragraph(f'Check Name: {check_result.name}')
                doc.add_paragraph(f'Description: {check_result.description}')
                doc.add_paragraph(f'Result: {check_result.result}')
                doc.add_paragraph(f'Message: {check_result.display()}')
                doc.add_paragraph('---' * 10)  # Divider
            
            # Save the Word document
            file_name = f'{selected_suite_name}_Report_{timestamp}.docx'
            doc.save(file_name)
            
            output.append_stdout(f"Results saved to {file_name}\n")
        except Exception as e:
            output.append_stdout(f"Error saving results: {str(e)}\n")

# Attach the "run suite" button to the handler function
run_button.on_click(run_selected_suite)

# -------------------- Step 6: Display the Jupyter Widget UI --------------------
# Layout for the widget
ui = VBox([
    suite_dropdown,  # Dropdown to select the suite
    test_selection_box,  # Checkboxes to select which tests to run
    threshold_slider,  # Slider to set test pass threshold
    run_button,  # Button to run the suite
    output  # Output display for test results and file save status
])

# Display the interactive UI
ui

In [28]:
ui

In [29]:
from ipywidgets import interactive, VBox, HBox, Dropdown, FloatSlider, Button, Checkbox, Output

available_suites = {
    'Data Integrity': data_integrity(),
    'Train-Test Validation': train_test_validation(),
    'Model Evaluation': model_evaluation()
}


# Checkboxes to select which tests to run within the suite
test_checkboxes = {test[0]: Checkbox(value=True, description=test[1].name()) 
                   for test in available_suites['Data Integrity'].checks.items()}

left_checkboxes = dict(list(test_checkboxes.items())[:6])
print(left_checkboxes)
right_checkboxes = dict(list(test_checkboxes.items())[6:])
print(right_checkboxes)
left_selection_box = VBox(list(left_checkboxes.values()))
right_selection_box = VBox(list(right_checkboxes.values()))


HBox([left_selection_box, right_selection_box])
                          


{0: Checkbox(value=True, description='Is Single Value'), 1: Checkbox(value=True, description='Special Characters'), 2: Checkbox(value=True, description='Mixed Nulls'), 3: Checkbox(value=True, description='Mixed Data Types'), 4: Checkbox(value=True, description='String Mismatch'), 5: Checkbox(value=True, description='Data Duplicates')}
{6: Checkbox(value=True, description='String Length Out Of Bounds'), 7: Checkbox(value=True, description='Conflicting Labels'), 8: Checkbox(value=True, description='Outlier Sample Detection'), 9: Checkbox(value=True, description='Feature Label Correlation'), 10: Checkbox(value=True, description='Feature Feature Correlation'), 11: Checkbox(value=True, description='Identifier Label Correlation')}


In [30]:
left_checkboxes

{0: Checkbox(value=True, description='Is Single Value'),
 1: Checkbox(value=True, description='Special Characters'),
 2: Checkbox(value=True, description='Mixed Nulls'),
 3: Checkbox(value=True, description='Mixed Data Types'),
 4: Checkbox(value=True, description='String Mismatch'),
 5: Checkbox(value=True, description='Data Duplicates')}

In [31]:
available_suites['Data Integrity'].checks[0]

IsSingleValue
	Conditions:
		0: Does not contain only a single value

In [32]:
type(available_suites['Data Integrity'].checks[0])

deepchecks.tabular.checks.data_integrity.is_single_value.IsSingleValue

In [33]:
available_suites['Data Integrity'].checks

OrderedDict([(0,
              IsSingleValue
              	Conditions:
              		0: Does not contain only a single value),
             (1,
              SpecialCharacters
              	Conditions:
              		0: Ratio of samples containing solely special character is less or equal to 0.1%),
             (2,
              MixedNulls
              	Conditions:
              		0: Number of different null types is less or equal to 1),
             (3,
              MixedDataTypes
              	Conditions:
              		0: Rare data types in column are either more than 10% or less than 1% of the data),
             (4,
              StringMismatch
              	Conditions:
              		0: No string variants),
             (5,
              DataDuplicates
              	Conditions:
              		0: Duplicate data ratio is less or equal to 5%),
             (6,
              StringLengthOutOfBounds
              	Conditions:
              		0: Ratio of string length outl

In [34]:
test_checkboxes = {test: test for test in available_suites['Data Integrity'].checks}
test_checkboxes

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11}

In [35]:
for test in enumerate(available_suites['Data Integrity'].checks):
    print(test)
    

(0, 0)
(1, 1)
(2, 2)
(3, 3)
(4, 4)
(5, 5)
(6, 6)
(7, 7)
(8, 8)
(9, 9)
(10, 10)
(11, 11)


In [36]:
import ipywidgets as widgets
from ipywidgets import VBox, Dropdown, Checkbox, Button, Output, IntSlider
from docx import Document
from datetime import datetime
from collections import OrderedDict
from deepchecks import Suite  # Assuming you are using Deepchecks

# Initialize global variables
test_checkboxes = {}

# Dropdown for selecting suite
suite_dropdown = widgets.Dropdown(
    options=['Data Integrity', 'Train-Test Validation', 'Model Evaluation'],
    value='Data Integrity',
    description='Select Suite:',
    disabled=False
)

# Available suites
available_suites = {
    'Data Integrity': Suite('Data Integrity Suite'),
    'Train-Test Validation': Suite('Train-Test Validation Suite'),
    'Model Evaluation': Suite('Model Evaluation Suite')
}

# Create UI components
test_selection_box = widgets.VBox()
threshold_slider = widgets.IntSlider(value=50, min=0, max=100, step=1, description='Threshold:')
run_button = widgets.Button(description='Run Suite', button_style='success', icon='check')
output = widgets.Output()

# Update test selection
def update_test_selection(change):
    global test_checkboxes
    selected_suite = change['new']
    test_checkboxes = {test[0]: Checkbox(value=True, description=test[1].name()) 
                       for test in available_suites[selected_suite].checks.items()}
    test_selection_box.children = list(test_checkboxes.values())

suite_dropdown.observe(update_test_selection, names='value')

def run_selected_suite(b):
    with output:
        output.clear_output()
        print("Running the suite...")

        # Example logic to run suite (simplified)
        output.append_stdout("Results saved to Word file.\n")

run_button.on_click(run_selected_suite)

# Display the interactive UI
ui = VBox([suite_dropdown, test_selection_box, threshold_slider, run_button, output])
ui


C:\Users\DELL\AppData\Local\Programs\Python\Python39\lib\site-packages\deepchecks\__init__.py:136: DeprecationWarning:

Ability to import base tabular functionality from the `deepchecks` package directly is deprecated, please import from `deepchecks.tabular` instead



In [37]:
# Step 1: Import necessary libraries
import ipywidgets as widgets
from ipywidgets import VBox, Dropdown, Checkbox, Button, Output, IntSlider

# Step 2: Create UI components
suite_dropdown = widgets.Dropdown(
    options=['Data Integrity', 'Train-Test Validation', 'Model Evaluation'],
    value='Data Integrity',
    description='Select Suite:'
)

test_selection_box = widgets.VBox([
    widgets.Checkbox(value=True, description='Test 1'),
    widgets.Checkbox(value=False, description='Test 2')
])

threshold_slider = widgets.IntSlider(
    value=50, 
    min=0, 
    max=100, 
    step=1, 
    description='Threshold:'
)

run_button = widgets.Button(
    description='Run Suite', 
    button_style='success', 
    icon='check'
)

output = widgets.Output()

# Step 3: Display UI
ui = VBox([suite_dropdown, test_selection_box, threshold_slider, run_button, output])
ui


In [38]:
# Step 1: Import necessary libraries
import ipywidgets as widgets
from ipywidgets import VBox, Dropdown, Checkbox, Button, Output, IntSlider

# Step 2: Create UI components
suite_dropdown = widgets.Dropdown(
    options=['Data Integrity', 'Train-Test Validation', 'Model Evaluation'],
    value='Data Integrity',
    description='Select Suite:'
)

# Dynamically change test checkboxes based on the selected suite
test_selection_box = widgets.VBox([
    widgets.Checkbox(value=True, description='Test 1'),
    widgets.Checkbox(value=False, description='Test 2')
])

# Slider to set threshold
threshold_slider = widgets.IntSlider(
    value=50, 
    min=0, 
    max=100, 
    step=1, 
    description='Threshold:'
)

# Button to run the suite
run_button = widgets.Button(
    description='Run Suite', 
    button_style='success', 
    icon='check'
)

# Output to display results
output = widgets.Output()

# Update UI when the suite selection changes
def update_test_selection(change):
    with output:
        output.clear_output()
        print(f"Suite changed to: {change['new']}")

# Attach observer to dropdown
suite_dropdown.observe(update_test_selection, names='value')

# Run suite logic
def run_selected_suite(b):
    with output:
        output.clear_output()
        selected_suite = suite_dropdown.value
        threshold_value = threshold_slider.value
        output.append_stdout(f"Running suite: {selected_suite} with threshold: {threshold_value}\n")
        output.append_stdout(f"Tests selected: {[checkbox.description for checkbox in test_selection_box.children if checkbox.value]}\n")

# Attach click event to run button
run_button.on_click(run_selected_suite)




In [39]:
# Step 3: Display UI
ui = VBox([suite_dropdown, test_selection_box, threshold_slider, run_button, output])
ui

In [40]:
import ipywidgets as widgets

widgets.IntSlider()


IntSlider(value=0)

In [41]:
# Step 1: Import necessary libraries
import ipywidgets as widgets
from IPython.display import display  # Added display to ensure UI renders
from ipywidgets import VBox, Dropdown, Checkbox, Button, Output, IntSlider

# Step 2: Create UI components
suite_dropdown = widgets.Dropdown(
    options=['Data Integrity', 'Train-Test Validation', 'Model Evaluation'],
    value='Data Integrity',
    description='Select Suite:'
)

# Test checkboxes
test_selection_box = widgets.VBox([
    widgets.Checkbox(value=True, description='Test 1'),
    widgets.Checkbox(value=False, description='Test 2')
])

# Slider to set threshold
threshold_slider = widgets.IntSlider(
    value=50, 
    min=0, 
    max=100, 
    step=1, 
    description='Threshold:'
)

# Button to run the suite
run_button = widgets.Button(
    description='Run Suite', 
    button_style='success', 
    icon='check'
)

# Output to display results
output = widgets.Output()

# Step 3: Display UI
ui = VBox([suite_dropdown, test_selection_box, threshold_slider, run_button, output])

# **Force display of the UI (important)**
display(ui)


In [42]:
from deepchecks.tabular.datasets.regression import wine_quality

data = wine_quality.load_data(data_format='Dataframe', as_train_test=False)
data.head(2)



,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.0,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.0,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5


In [43]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, :-1], data['quality'], test_size=0.2, random_state=42)
gbr = GradientBoostingRegressor()
gbr.fit(X_train, y_train)

GradientBoostingRegressor()

In [44]:
from deepchecks.tabular import Dataset

# Categorical features can be heuristically inferred, however we
# recommend to state them explicitly to avoid misclassification.

# Metadata attributes are optional. Some checks will run only if specific attributes are declared.

train_ds = Dataset(X_train, label=y_train, cat_features=[])
test_ds = Dataset(X_test, label=y_test, cat_features=[])

In [45]:
from deepchecks.tabular.suites import model_evaluation

evaluation_suite = model_evaluation()
suite_result = evaluation_suite.run(train_ds, test_ds, gbr)
# Note: the result can be saved as html using suite_result.save_as_html()
# or exported to json using suite_result.to_json()
suite_result.show()

Accordion(children=(VBox(children=(HTML(value='\n<h1 id="summary_DHRVSOVJJFL37SJAFIWUV0GLO">Model Evaluation S…

In [46]:
from deepchecks.tabular.suites import data_integrity
from deepchecks.checks import MixedNulls  # Import the correct class for type checking

# Step 1: Create the Data Integrity Suite
suite = data_integrity()

# Step 2: Print the list of checks to verify keys, names, and types
print("Available checks in the Data Integrity Suite:")
for i, (name, check) in enumerate(suite.checks.items()):
    print(f"Check {i}: Key = {name}, Type = {type(check).__name__}")



Available checks in the Data Integrity Suite:
Check 0: Key = 0, Type = IsSingleValue
Check 1: Key = 1, Type = SpecialCharacters
Check 2: Key = 2, Type = MixedNulls
Check 3: Key = 3, Type = MixedDataTypes
Check 4: Key = 4, Type = StringMismatch
Check 5: Key = 5, Type = DataDuplicates
Check 6: Key = 6, Type = StringLengthOutOfBounds
Check 7: Key = 7, Type = ConflictingLabels
Check 8: Key = 8, Type = OutlierSampleDetection
Check 9: Key = 9, Type = FeatureLabelCorrelation
Check 10: Key = 10, Type = FeatureFeatureCorrelation
Check 11: Key = 11, Type = IdentifierLabelCorrelation


In [47]:
# Step 3: Access the MixedNulls check using its numeric key (2)
if 2 in suite.checks:  # Check that key 2 exists
    mixed_nulls_check = suite.checks[2]
else:
    raise ValueError("MixedNulls check not found in the Data Integrity suite.")

# Step 4: Clear the existing conditions (if any)
mixed_nulls_check.conditions = {}  # Clears all existing conditions

# Step 5: Add a new condition with a customized threshold
mixed_nulls_check.add_condition('proportion of mixed nulls is less than', threshold=0.1)

# Verify the updated conditions
print("\nUpdated conditions for MixedNulls Check:")
print(mixed_nulls_check.conditions)


TypeError: add_condition() missing 1 required positional argument: 'condition_func'

In [ ]:
from deepchecks.tabular.suites import data_integrity
from deepchecks.checks import MixedNulls  # Import the correct class for type checking

# Step 1: Create the Data Integrity Suite
suite = data_integrity()

# Step 2: Print the list of checks to verify keys, names, and types
print("Available checks in the Data Integrity Suite:")
for i, (name, check) in enumerate(suite.checks.items()):
    print(f"Check {i}: Key = {name}, Type = {type(check).__name__}")

# Step 3: Access the MixedNulls check using its numeric key (2)
if 2 in suite.checks:  # Check that key 2 exists
    mixed_nulls_check = suite.checks[2]
else:
    raise ValueError("MixedNulls check not found in the Data Integrity suite.")

# Step 4: Clear the existing conditions (if any)
mixed_nulls_check.conditions = {}  # Clears all existing conditions

# Step 5: Add a new condition with a customized threshold
# The condition checks if the result's value (proportion of mixed nulls) is less than 0.1
mixed_nulls_check.add_condition(
    'proportion of mixed nulls is less than 0.1', 
    lambda result: result.value < 0.1
)

# Verify the updated conditions
print("\nUpdated conditions for MixedNulls Check:")
print(mixed_nulls_check.conditions)


In [ ]:
mixed_nulls_check